In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [2]:
# Loop through cities
tweet_dir= Path("../data/processed/park_user_tweets")
cities = list(tweet_dir.glob("*.json"))

## load all stop wordds
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)

## load park type file
park_info = pd.read_csv('../data/processed/tpl/park_info_size.csv',dtype=str)

cats = ['acres_1_10', 'acres_gt_100', 'acres_10_100', 'acres_lt_1']

In [3]:
park_tweets_bytype = {cat:[] for cat in cats}
time_tweets_bytype = {cat:[] for cat in cats}
park_stops = []

In [4]:
park_info.head()

,ParkID,Park_Place,Park_Pla_1,Park_Desig,Park_Size_,size_cat,park_type
0,24040000001,Baltimore city,2404000,LP,0.168728709898,acres_lt_1,local_park
1,24040000002,Baltimore city,2404000,LP,0.622870976624,acres_lt_1,local_park
2,24040000003,Baltimore city,2404000,LP,1.3572710309,acres_1_10,local_park
3,24040000004,Baltimore city,2404000,LP,0.992805422216,acres_lt_1,local_park
4,24040000005,Baltimore city,2404000,LP,1.72528447101,acres_1_10,local_park


In [11]:
parkcols = park_info[['ParkID', 'size_cat']].copy()
parkcols.columns = ['park_id','size_cat']
parkcols.head()

,park_id,size_cat
0,24040000001,acres_lt_1
1,24040000002,acres_lt_1
2,24040000003,acres_1_10
3,24040000004,acres_lt_1
4,24040000005,acres_1_10


In [12]:
for city in cities:
    park_stops = []
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    tweets_w_parks = get_time_control_text_park(park_user_tweets)
    tweet_df = pd.DataFrame.from_dict(tweets_w_parks,dtype=str)
    tweet_df = pd.merge(tweet_df,parkcols,how='left',on='park_id')
    cats = list(set(tweet_df.size_cat.values))
    park_list = list(set(tweet_df.park_name))
    park_stops += get_park_stopwords(park_list)
    for cat in cats:
        tweet_subset = tweet_df[tweet_df.size_cat == cat]
        n = min(500, len(tweet_subset))
        print("City: {} -{} {} tweets".format(city, cat, n))
        index_list = range(len(tweet_subset))
        sample = random.sample(index_list, n)
        park_text = list(tweet_subset.park_text.values)
        control_text = list(tweet_subset.control_text.values)
        park_tweet_sample = [park_text[i] for i in sample]
        control_tweet_sample = [control_text[i] for i in sample]
        park_tweets_bytype[cat] += park_tweet_sample
        time_tweets_bytype[cat] += control_tweet_sample

City: ../data/processed/park_user_tweets/CO_Denver_0820000.json -acres_gt_100 500 tweets
City: ../data/processed/park_user_tweets/CO_Denver_0820000.json -acres_1_10 500 tweets
City: ../data/processed/park_user_tweets/CO_Denver_0820000.json -acres_lt_1 33 tweets
City: ../data/processed/park_user_tweets/CO_Denver_0820000.json -acres_10_100 500 tweets
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json -acres_gt_100 500 tweets
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json -acres_1_10 500 tweets
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json -acres_lt_1 6 tweets
City: ../data/processed/park_user_tweets/AZ_Phoenix_0455000.json -acres_10_100 500 tweets
City: ../data/processed/park_user_tweets/FL_Jacksonville_1235000.json -acres_gt_100 500 tweets
City: ../data/processed/park_user_tweets/FL_Jacksonville_1235000.json -acres_1_10 500 tweets
City: ../data/processed/park_user_tweets/FL_Jacksonville_1235000.json -acres_lt_1 38 tweets
City: ../data/p

In [13]:
# check length of < 1 ..park_tweets_bytype = {cat:[] for cat in cats}
for park_type,tweets in park_tweets_bytype.items():
    print(park_type,len(tweets))

len(park_stops)

acres_1_10 12052
acres_gt_100 12225
acres_10_100 12500
acres_lt_1 6473


577

In [14]:
##save each thing to a sample folder with name of date sample was run (user control tweet may change each time)
today = "0513"
results_dir = Path("../data/processed/combined_tweets/tweet_samples_{}".format(today))
with open(results_dir/Path("park_tweets_bytype.json"), 'w') as fp:
    json.dump(park_tweets_bytype,fp)
with open(results_dir/Path("time_tweets_bytype.json"), 'w') as fp:
    json.dump(time_tweets_bytype,fp)


In [38]:
## load frequent stop words  stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)
    
#load stop words selected from raw wordshifts for probelmatic words
freq_stops = []
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

# combine with  names of parks
all_stops = freq_stops + park_stops


# put through ngram filter, lower
all_stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(all_stops), path='../src/func/ngrams.bin')).keys()]))
print(len(all_stops))
# select ones with valence below 4 or greater than 6
stops_w_val=[]
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
            #print(stop,senti_dict[stop])
# check how #many of stop words are in labmt outside 4-6 range
print(len(stops_w_val))

615
116


In [41]:
results_dict = {}

for cat in cats:
    results_dict[cat] = bootstrap_sentiment(park_tweets_bytype[cat], time_tweets_bytype[cat], stops_w_val,sample=.8,runs=5)

In [42]:
results_dict

{'acres_1_10': [0.11472264350773287,
  0.12461761098934154,
  0.12551167431450327,
  0.12805631864760603,
  0.12252351593659228],
 'acres_gt_100': [0.13084495336498048,
  0.12705553664694236,
  0.12880077497983056,
  0.13220354221293285,
  0.12786786043383636],
 'acres_10_100': [0.07961949429900717,
  0.07756543958249118,
  0.08337921863665088,
  0.06752940800897456,
  0.08438186476866871],
 'acres_lt_1': [0.09425190880783951,
  0.1026447778398003,
  0.10307146959840985,
  0.1039210558804804,
  0.09764765382976748]}

In [44]:
with open('results/city_combined_0421/bootstrap_parktype.json','w') as fp:
    json.dump(results_dict, fp)

In [45]:
results_dict_freq = {}

for cat in cats:
    results_dict_freq[cat] = bootstrap_sentiment(park_tweets_bytype[cat], time_tweets_bytype[cat], freq_stops,sample=.8,runs=5)

In [1]:
results_dict_freq

NameError: name 'results_dict_freq' is not defined